In [1]:
%pip install pandas
%pip install pyarrow
%pip install sleeper-py

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sleeperpy import *

In [39]:
base_s3_path = 's3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper'
past_league_ids = ['541384381865115648', '706690303247065088','784536701455429632','961113588070985728']
for index, league_id in enumerate(past_league_ids):
    year = index + 2020
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    for week in range(1, 25):
        matchups = pd.DataFrame(Leagues.get_matchups(league_id, week))
        if not matchups.empty:
            week_result = pd.merge(rosters, matchups, on='roster_id', how='inner')
            players_points_list = []
            for players_points_week in week_result['players_points'].to_list():
                players_points_list.append([{'player_id': player_id, 'points': points} for player_id, points in players_points_week.items()])
            week_result['players_points'] = players_points_list
            week_result = week_result.drop(columns=['settings', 'metadata'])
            week_result.to_parquet(f'{base_s3_path}/matchups/{year}/{week}/result.parquet', engine='pyarrow')

In [40]:
matchups = pd.read_parquet('s3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper/matchups/')

In [41]:
print(matchups)

   co_owners       keepers           league_id            owner_id player_map  \
0       None  [4137, 4881]  541384381865115648  463128802726244352       None   
1       None          None  541384381865115648  586272662465060864       None   
2       None  [3242, 4029]  541384381865115648   86916165133549568       None   
3       None          None  541384381865115648  602974973434343424       None   
4       None        [5012]  541384381865115648  601103930037944320       None   
..       ...           ...                 ...                 ...        ...   
7       None  [1426, 7611]  961113588070985728  600100762684563456       None   
8       None        [2374]  961113588070985728  541051698865418240       None   
9       None        [7525]  961113588070985728  586300618197475328       None   
10      None          None  961113588070985728  541054017910829056       None   
11      None        [6786]  961113588070985728  540275269586792448       None   

                           